In [50]:
import torch
from torch import Tensor
import numpy as np
from torch import nn
import dlc_bci as bci
from torch.autograd import Variable
from torch import optim
from torch.nn import functional as F
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import model_selection

In [51]:
train_input, train_target = bci.load(root = './data_bci')
X = train_input.numpy()
y = train_target.numpy()
kfolds = model_selection.KFold(n_splits=3, random_state=1234, shuffle=True)

In [52]:
def compute_nb_errors(model, data_input, data_target, mini_batch_size):
    nb_data_errors = 0

    for b in range(0, data_input.size(0), mini_batch_size):
        if b + mini_batch_size <= data_input.size(0):
            output = model(data_input.narrow(0, b, mini_batch_size))
            _, predicted_classes = torch.max(output.data, 1)
            for k in range(0, mini_batch_size):
                if data_target.data[b + k] != predicted_classes[k]:
                    nb_data_errors = nb_data_errors + 1
        else:       
            output = model(data_input.narrow(0, b, data_input.size(0) - b))
            _, predicted_classes = torch.max(output.data, 1)
            for k in range(0, data_input.size(0) - b):
                if data_target.data[b + k] != predicted_classes[k]:
                    nb_data_errors = nb_data_errors + 1

    return nb_data_errors

In [53]:
def train_model_full(network_model, param, train_input, train_target, kfolds, nb_epochs, lambdda = 0.01, lr = 0.001):
    
    acc_train_kfold = []
    loss_train_kfold = []
    acc_val_kfold = []
    loss_val_kfold = []
    
    for train_index, val_index in kfolds.split(X):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        X_train = Variable(torch.from_numpy(X_train))
        X_val = Variable(torch.from_numpy(X_val))
        y_train = Variable(torch.from_numpy(y_train))
        y_val = Variable(torch.from_numpy(y_val)) 
        
        model = network_model(**param)
        
        loss_train, loss_val, acc_train, acc_val = train_model(model, 
                                                               X_train, 
                                                               y_train,
                                                               X_val, 
                                                               y_val, 
                                                               kfolds,
                                                               nb_epochs, 
                                                               lambdda, 
                                                               lr)
        acc_train_kfold.append(acc_train)
        loss_train_kfold.append(loss_train)
        acc_val_kfold.append(acc_val)
        loss_val_kfold.append(loss_val)
        
    acc_train_kfold = np.mean(np.array(acc_train_kfold), axis=0)
    acc_val_kfold = np.mean(np.array(acc_val_kfold), axis=0)

    loss_train_kfold = np.mean(np.array(loss_train_kfold), axis=0)
    loss_val_kfold = np.mean(np.array(loss_val_kfold), axis=0)
    
    print('\n\n---- Epochs Done -----\n')
    print('Loss: train ~ {} Acc train ~ {} \nLoss: val ~ {} / Acc val ~ {}\n'
         .format(round(loss_train_kfold[-1], 3), 
                 round(acc_train_kfold[-1], 3), 
                 round(loss_val_kfold[-1], 3), 
                 round(acc_val_kfold[-1], 3)))
    return loss_train_kfold, loss_val_kfold, acc_train_kfold, acc_val_kfold

def train_model(model, X_train, y_train, X_val, y_val, kfolds, nb_epochs, lambdda = 0.01, lr = 0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr)
    
    acc_train = []
    acc_val = []
    loss_train = []
    loss_val = []
        
    for e in range(0, nb_epochs):
         
        model.train(True)
        for b in list(range(0, X_train.size(0), mini_batch_size)):
            if b + mini_batch_size <= X_train.size(0):
                output = model(X_train.narrow(0, b, mini_batch_size))
                loss = criterion(output, y_train.narrow(0, b, mini_batch_size))
            else:
                output = model(X_train.narrow(0, b, X_train.size(0) - b))
                loss = criterion(output, y_train.narrow(0, b, X_train.size(0) - b))

            for p in model.parameters():
                loss += lambdda*p.pow(2).sum()
            model.zero_grad()
            loss.backward()
            optimizer.step()
                
            model.train(False)
            output_train = model(X_train)
            output_val = model(X_val)
            
        acc_val.append(1-compute_nb_errors(model, X_val, y_val, mini_batch_size=mini_batch_size)/X_val.size(0))
        acc_train.append(1-compute_nb_errors(model, X_train, y_train, mini_batch_size=mini_batch_size)/X_train.size(0))
        loss_train.append(criterion(output_train, y_train).data[0])
        loss_val.append(criterion(output_val, y_val).data[0])

    return loss_train, loss_val, acc_train, acc_val

In [54]:
class Conv_net(nn.Module):
    def __init__(self, layers, layers_conv, kernel_size, pooling_kernel_size, p):
        super(Conv_net, self).__init__()
        self.pooling_kernel_size = pooling_kernel_size
        self.additional_conv_hidden = nn.ModuleList()
        self.additional_fc_hidden = nn.ModuleList()
        self.droput_layers = nn.ModuleList()
        self.batch_normalization = nn.ModuleList()
        
        for l in range(len(layers_conv)-1):
            self.additional_conv_hidden.append(nn.Conv1d(layers_conv[l], layers_conv[l+1], kernel_size=kernel_size[l]))
            self.droput_layers.append(torch.nn.Dropout(p=p[l]))
            self.batch_normalization.append(torch.nn.BatchNorm1d(layers_conv[l+1]))
        size = train_input.shape[2]

        for i in range(len(kernel_size)):
            size-=(kernel_size[i]-1)

            size//=pooling_kernel_size[i]

        self.additional_fc_hidden.append(nn.Linear(size*layers_conv[-1], layers[0]))
        self.droput_layers.append(torch.nn.Dropout(p=p[l+1]))
        self.batch_normalization.append(torch.nn.BatchNorm1d(layers[0]))
        self.flat_size = size*layers_conv[-1]
        
        start_p = l+2

        for l in range(len(layers)-1):
            self.additional_fc_hidden.append(nn.Linear(layers[l], layers[l+1]))
            if l != len(layers)-2:
                self.droput_layers.append(torch.nn.Dropout(p=p[l+start_p]))
                self.batch_normalization.append(torch.nn.BatchNorm1d(layers[l+1]))

    def forward(self, x):
        for l in range(len(self.additional_conv_hidden)):
            x = self.droput_layers[l](self.batch_normalization[l](F.relu(F.max_pool1d(self.additional_conv_hidden[l](x), \
                                                          kernel_size=self.pooling_kernel_size[l]))))
        x=x.view(-1, self.flat_size)
        for l in range(len(self.additional_fc_hidden)-1):
            index = len(self.additional_conv_hidden)+l
            x = self.droput_layers[index](self.batch_normalization[index](F.relu(self.additional_fc_hidden[l](x))))
        x = self.additional_fc_hidden[-1](x)
        return x

In [55]:
p_list = [0.2, 0.2, 0]
layers = [5, 2]
layers_conv = [28, 4, 4]
kernel_size = [6, 6]
pooling_kernel_size = [3, 2]
    
parameters = {'layers': layers, 
              'layers_conv': layers_conv, 
              'kernel_size': kernel_size, 
              'pooling_kernel_size': pooling_kernel_size, 
              'p': p_list}
    
mini_batch_size = 79
nb_epochs = 100
costs, costs_val, acc, acc_val = train_model_full(Conv_net, 
                                                  parameters,
                                                  train_input, 
                                                  train_target, 
                                                  kfolds, 
                                                  nb_epochs,
                                                  lambdda=0.0375)



---- Epochs Done -----

Loss: train ~ 0.562 Acc train ~ 0.684 
Loss: val ~ 0.716 / Acc val ~ 0.541



In [97]:
class CNN(nn.Module):
    def __init__(self, C, K, V, S, F):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(             # input shape (1, S, V)
                in_channels=1,              # input channels
                out_channels=C,             # number of filters
                kernel_size=(K,V),          # filter size
                padding=(1,0),              # if want same width and length of this image after con2d, padding=(kernel_size-1)/2 if stride=1
        )                                   # output shape (C, S, 1)
        self.relu = nn.ReLU()               # ReLU activation
        self.max_pool = nn.MaxPool1d(S)     # max-pool each filter into 1 output
        self.out = nn.Linear(C, F)          # fully connected layer, output F classes

    def forward(self, x):
        out = x.unsqueeze(1)
        out = self.conv1(out)
        out = self.relu(out).squeeze(1)
        print(out.shape)
        out = self.max_pool(out)#.squeeze(2).float()
        print(out.shape)
        out = self.out(out)
        return out

In [98]:
parameters = {'C': 28, 'S': 1, 'K': 3, 'V': 50, 'F': 2}
mini_batch_size = 79
nb_epochs = 100
costs, costs_val, acc, acc_val = train_model_full(CNN, 
                                                  parameters, 
                                                  train_input, 
                                                  train_target, 
                                                  kfolds,
                                                  nb_epochs, 
                                                  lambdda=0.0375)

torch.Size([79, 28, 28, 1])


ValueError: expected 3D input (got 4D input)